In [61]:
class Block:
    
    def __init__(self,name):
        self.name = name
    
    def render(self, i):
        i += 1
        return "X{}[{}]".format(i, self.name), i
        
class ParallelBlock:
    def __init__(self, name, components):
        self.name = name
        self.components
        
class StackBlock:
    def __init__(self, name, components):
        self.name = name
        self.components = components
        
    def render(self, i):
        start_i = i
        out = ""
        
        # Render components
        for component in self.components:
            component, i = component.render(i)
            out += component
            out += "\n"
        end_i = i
        
        # Render connections
        for ix in range(start_i+1, end_i):
            out += "X{}-->X{}".format(ix, ix+1)
            out += "\n"
        
        # Subgraph it
        i += 1
        out = "subgraph X{} \n".format(i) + out
        out += "end"
        
        return out, i

In [62]:
block_a = Block(name='A')
block_b = Block(name='B')
block_c = Block(name='C')
simulation_block = StackBlock('Simulation 1', [block_a, block_b, block_c])

In [63]:
i = 0
g, i = simulation_block.render(i)
print(g)

subgraph X4 
X1[A]
X2[B]
X3[C]
X1-->X2
X2-->X3
end


```mermaid
graph LR
A[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]
    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]

```

In [19]:
from IPython.display import display

display("""```mermaid
graph LR
A[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]
    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]

```""")

'```mermaid\ngraph LR\nA[JSON Object \n\n Each spec has a repo for tracking changes \n Must conform to the json specification \n Defines all aspects of the spec including blocks, spaces and actions] -->B[MSML Object \n\n JSON file and parsed, with validations and mappings along the way \n Can show different views on the fly]\n    B --> C[Report Outputs \n\n Automatically build reports for the full spec or subviews \n Example: all blocks with an effect on variable XYZ]\n\n```'